## Day 28 Lecture 2 Assignment

In this assignment, we will learn about overfitting and regularization. We will use the king county housing dataset loaded below and analyze the regression from this dataset.

In [0]:
%matplotlib inline

import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse



In [0]:
king = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/kc_house_data.csv')

In [0]:
king_county = king.copy()
king_county.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [0]:
king_county['log_price']= np.log(king_county['price'])
king_county['log_sqft_living']= np.log(king_county['sqft_living'])

In [0]:
king_county.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,log_price,log_sqft_living
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,12.309982,7.073270
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,13.195614,7.851661
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,12.100712,6.646391
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,13.311329,7.580700
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,13.142166,7.426549


Perform the same transformations in the previous assignment to meet model assumptions:
1. Transform price and sqft_living by applying the natual log
2. Remove all columns except: log price, bedrooms, bathrooms, log_sqft_living, floors, waterfront
3. Split the data into train and test subsets. 20% of the data should be in the test subset

In [0]:
king_county.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 23 columns):
id                 21613 non-null int64
date               21613 non-null object
price              21613 non-null float64
bedrooms           21613 non-null int64
bathrooms          21613 non-null float64
sqft_living        21613 non-null int64
sqft_lot           21613 non-null int64
floors             21613 non-null float64
waterfront         21613 non-null int64
view               21613 non-null int64
condition          21613 non-null int64
grade              21613 non-null int64
sqft_above         21613 non-null int64
sqft_basement      21613 non-null int64
yr_built           21613 non-null int64
yr_renovated       21613 non-null int64
zipcode            21613 non-null int64
lat                21613 non-null float64
long               21613 non-null float64
sqft_living15      21613 non-null int64
sqft_lot15         21613 non-null int64
log_price          21613 non-null float6

In [0]:
king_county.drop(['id', 'date', 'sqft_living',
       'sqft_lot', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'price' ], axis=1, inplace=True)

In [0]:
king_county.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 6 columns):
bedrooms           21613 non-null int64
bathrooms          21613 non-null float64
floors             21613 non-null float64
waterfront         21613 non-null int64
log_price          21613 non-null float64
log_sqft_living    21613 non-null float64
dtypes: float64(4), int64(2)
memory usage: 1013.2 KB


In [0]:
# answer below:

Y= king_county['log_price']

X= king_county[['bedrooms', 'bathrooms', 'floors', 'waterfront', 'log_sqft_living']]

X_train, X_test, y_train, y_test= train_test_split(X, Y, test_size = 0.2, random_state = 465)


Apply a ridge regression model with lambda=50 to the data and evaluate by looking at r squared for test and train

Perform a grid search for the following values of alpha: 1, 5, 10, 100, 500, 1000, 10000 to find the most optimal ridge regression model.

In [0]:
# answer below:
alphas = [50]

ridge_cv = RidgeCV(alphas=alphas, cv=10)
ridge_cv.fit(X_train, y_train)

# We are making predictions here
y_preds_train = ridge_cv.predict(X_train)
y_preds_test = ridge_cv.predict(X_test)

print("RidgeCV() regression test")
print("Best alpha value is: {}".format(ridge_cv.alpha_))
print("R-squared of the model in training set is: {}".format(ridge_cv.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(ridge_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))


RidgeCV() regression test
Best alpha value is: 50
R-squared of the model in training set is: 0.485494107558357
-----Test set statistics-----
R-squared of the model in test set is: 0.46944033145661235
Mean absolute error of the prediction is: 0.3095391158491194
Mean squared error of the prediction is: 0.14666604385123422
Root mean squared error of the prediction is: 0.3829700299647927
Mean absolute percentage error of the prediction is: 2.3811942543162834


In [0]:
# answer below:
#alphas = [np.power(50.0,p) for p in np.arange(-10,40,1)]
alphas = [1, 5, 10, 100, 500, 1000, 10000]

ridge_cv = RidgeCV(alphas=alphas, cv=10)
ridge_cv.fit(X_train, y_train)

# We are making predictions here
y_preds_train = ridge_cv.predict(X_train)
y_preds_test = ridge_cv.predict(X_test)

print("RidgeCV() regression test")
print("Best alpha value is: {}".format(ridge_cv.alpha_))
print("R-squared of the model in training set is: {}".format(ridge_cv.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(ridge_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))


RidgeCV() regression test
Best alpha value is: 1
R-squared of the model in training set is: 0.48670949629179405
-----Test set statistics-----
R-squared of the model in test set is: 0.4709140358772267
Mean absolute error of the prediction is: 0.3089280944772293
Mean squared error of the prediction is: 0.14625865819794667
Root mean squared error of the prediction is: 0.38243778343404655
Mean absolute percentage error of the prediction is: 2.3770780928061694


In [0]:
# answer below:
alphas = [np.power(50.0,p) for p in np.arange(-10,40,1)]

ridge_cv = RidgeCV(alphas=alphas, cv=10)
ridge_cv.fit(X_train, y_train)

# We are making predictions here
y_preds_train = ridge_cv.predict(X_train)
y_preds_test = ridge_cv.predict(X_test)

print("RidgeCV() regression test")
print("Best alpha value is: {}".format(ridge_cv.alpha_))
print("R-squared of the model in training set is: {}".format(ridge_cv.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(ridge_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))


RidgeCV() regression test
Best alpha value is: 0.02
R-squared of the model in training set is: 0.4867103230497562
-----Test set statistics-----
R-squared of the model in test set is: 0.4709274687267148
Mean absolute error of the prediction is: 0.30891719635561565
Mean squared error of the prediction is: 0.14625494486840263
Root mean squared error of the prediction is: 0.38243292858801087
Mean absolute percentage error of the prediction is: 2.3770061867318613
